In [1]:
# define imports
import os
import json
import shutil
import numpy as np
import pandas as pd

In [2]:
# define which datasets
model_name = '2021_03_15-Full_Model'
export_name = 'export_test'
steps = [1, 3, 5, 6, 7] # [1, 3, 5, 7] # 1-7
target_strains = [] # all, if empty
target_cells = ['H0R'] # all, if empty

In [3]:
# create folder structure
if not os.path.exists('exports'):
    os.makedirs('exports')
export_folderpath = os.path.join('exports', export_name)
if os.path.exists(export_folderpath) and os.path.isdir(export_folderpath):
    shutil.rmtree(export_folderpath)
os.makedirs(export_folderpath)

In [4]:
def create_step_12_excel(data, export_filepath, filename):
    cell = {}
    for strain in data:
        if target_strains and strain not in target_strains:
            continue
            
        for pos in data[strain]:
            for cell_type in ['seam_cells', 'annotations']:
                for cell_name in data[strain][pos][cell_type]:
                    if target_cells and cell_name not in target_cells:
                        continue
                        
                    cleaned_cell_name = cell_name.replace('/','-')
                    if cleaned_cell_name not in cell:
                        cell[cleaned_cell_name] = []
                    
                    new_coord_list = []
                    for idx, new_tp in enumerate(data[strain][pos][cell_type][cell_name]['timepoints']):
                        new_fmt = {}
                        if idx == 0:
                            new_fmt['info'] = strain
                        if idx == 1:    
                            new_fmt['info'] = pos
                            
                        new_fmt['timepoint'] = data[strain][pos][cell_type][cell_name]['timepoints'][idx]
                        new_fmt['x'] = data[strain][pos][cell_type][cell_name]['coordinates'][idx][0]
                        new_fmt['y'] = data[strain][pos][cell_type][cell_name]['coordinates'][idx][1]
                        new_fmt['z'] = data[strain][pos][cell_type][cell_name]['coordinates'][idx][2]
                        new_coord_list.append(new_fmt)
                        
                    cell[cleaned_cell_name].append(new_coord_list)
    
    # create excel
    writer = pd.ExcelWriter(os.path.join(
        export_filepath, '{}.xlsx'.format(filename)), engine='xlsxwriter')
    for cell_name in cell:
        col = 0
        for cell_data in cell[cell_name]:
            cell_data_df = pd.DataFrame(cell_data)
            cell_data_df.to_excel(writer, sheet_name=cell_name, startrow=0, startcol=col, index=False)   
            col = col + cell_data_df.shape[1] + 1
    writer.save()    


def create_step_35_excel(data, export_filepath, filename):
    cell = {}
    for strain in data:
        if target_strains and strain not in target_strains:
            continue
            
        for pos in data[strain]:
            for cell_type in ['seam_cells', 'annotations']:
                for cell_name in data[strain][pos][cell_type]:
                    if target_cells and cell_name not in target_cells:
                        continue
                        
                    cleaned_cell_name = cell_name.replace('/','-')
                    if cleaned_cell_name not in cell:
                        cell[cleaned_cell_name] = []
                    
                    new_coord_list = []
                    for idx, new_tp in enumerate(data[strain][pos][cell_type][cell_name]['timepoints']):
                        new_fmt = {}
                        if idx == 0:
                            new_fmt['info'] = strain
                        if idx == 1:    
                            new_fmt['info'] = pos
                            
                        new_fmt['timepoint'] = data[strain][pos][cell_type][cell_name]['timepoints'][idx]
                        new_fmt['x'] = data[strain][pos][cell_type][cell_name]['coordinates'][idx][0]
                        new_fmt['y'] = data[strain][pos][cell_type][cell_name]['coordinates'][idx][1]
                        new_fmt['z'] = data[strain][pos][cell_type][cell_name]['coordinates'][idx][2]
                        new_coord_list.append(new_fmt)
                        
                    cell[cleaned_cell_name].append(new_coord_list)
    
    # create excel
    writer = pd.ExcelWriter(os.path.join(
        export_filepath, '{}.xlsx'.format(filename)), engine='xlsxwriter')
    for cell_name in cell:
        col = 0
        for cell_data in cell[cell_name]:
            cell_data_df = pd.DataFrame(cell_data)
            cell_data_df.to_excel(writer, sheet_name=cell_name, startrow=0, startcol=col, index=False)   
            col = col + cell_data_df.shape[1] + 1
    writer.save()  

    
def create_step_67_excel(data, export_filepath, filename):
    cell = {} 
    for cell_type in ['seam_cells', 'annotations']:
        for cell_name in data[cell_type]:
            if target_cells and cell_name not in target_cells:
                continue

            cleaned_cell_name = cell_name.replace('/','-')
            if cleaned_cell_name not in cell:
                cell[cleaned_cell_name] = []

            new_coord_list = []
            for idx, new_tp in enumerate(range(len(data[cell_type][cell_name]))):
                new_fmt = {}
                new_fmt['timepoint'] = idx #, data[cell_type][cell_name]['timepoints'][idx]
                new_fmt['x'] = data[cell_type][cell_name][idx][0]
                new_fmt['y'] = data[cell_type][cell_name][idx][1]
                new_fmt['z'] = data[cell_type][cell_name][idx][2]
                new_coord_list.append(new_fmt)

            cell[cleaned_cell_name].append(new_coord_list)
    
    # create excel
    writer = pd.ExcelWriter(os.path.join(
        export_filepath, '{}.xlsx'.format(filename)), engine='xlsxwriter')
    for cell_name in cell:
        col = 0
        for cell_data in cell[cell_name]:
            cell_data_df = pd.DataFrame(cell_data)
            cell_data_df.to_excel(writer, sheet_name=cell_name, startrow=0, startcol=col, index=False)   
            col = col + cell_data_df.shape[1] + 1
    writer.save()  

In [5]:
# load in data
folderpath = os.path.join(os.getcwd(), 'workspace', model_name)
for step in steps:
    step_names = [
        '1_compiled_data', 
        '2_compiled_data_no_outliers', 
        '3_compiled_data_interpolation', 
        '4_seam_cell_warping_model', # '4_seam_cell_warping_model_unsmoothed',
        '5_compiled_data_warped',
        '6_cell_coord_stats_by_timepoint',
        '7_cell_coord_stats_by_timepoint_smoothed'
    ]
    with open('config.json') as f:
        config = json.load(f)
        cell_info_filepath = config['settings']['mipav_output']['cell_info']
        show_labels = config['settings']['mipav_output']['labels_on']

    filename = step_names[step-1]
    filepath = os.path.join(folderpath, filename + '.json')

    with open(filepath) as f:
        step_data = json.load(f)

    if step in [1, 2]:
        create_step_12_excel(step_data, export_folderpath, filename)
    elif step in [3, 4, 5]:
        create_step_35_excel(step_data, export_folderpath, filename)
    elif step in [6, 7]:
        create_step_67_excel(step_data['mean'], export_folderpath, filename)